# Regiment

### Introduction:

Special thanks to: http://chrisalbon.com/ for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,317 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,687 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

### Step 2. Create the DataFrame with the following values:

In [2]:
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'],
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'],
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'],
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}

### Step 3. Assign it to a variable called regiment.
#### Don't forget to name each column

In [4]:
import pandas as pd
df = pd.DataFrame(raw_data)

In [12]:
raw_data = {
    'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'],
    'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd'],
    'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'],
    'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
    'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]
}

regiments = spark.createDataFrame(list(zip(*raw_data.values())), schema=list(raw_data.keys()))

regiments.show()

+----------+-------+--------+------------+-------------+
|  regiment|company|    name|preTestScore|postTestScore|
+----------+-------+--------+------------+-------------+
|Nighthawks|    1st|  Miller|           4|           25|
|Nighthawks|    1st|Jacobson|          24|           94|
|Nighthawks|    2nd|     Ali|          31|           57|
|Nighthawks|    2nd|  Milner|           2|           62|
|  Dragoons|    1st|   Cooze|           3|           70|
|  Dragoons|    1st|   Jacon|           4|           25|
|  Dragoons|    2nd|  Ryaner|          24|           94|
|  Dragoons|    2nd|    Sone|          31|           57|
|    Scouts|    1st|   Sloan|           2|           62|
|    Scouts|    1st|   Piger|           3|           70|
|    Scouts|    2nd|   Riani|           2|           62|
|    Scouts|    2nd|     Ali|           3|           70|
+----------+-------+--------+------------+-------------+



### Step 4. What is the mean preTestScore from the regiment Nighthawks?  

In [13]:
regiments.show(1)

+----------+-------+------+------------+-------------+
|  regiment|company|  name|preTestScore|postTestScore|
+----------+-------+------+------------+-------------+
|Nighthawks|    1st|Miller|           4|           25|
+----------+-------+------+------------+-------------+
only showing top 1 row



In [15]:
regiments.groupBy('regiment').avg('preTestScore').filter(regiments.regiment == 'Nighthawks').show()

+----------+-----------------+
|  regiment|avg(preTestScore)|
+----------+-----------------+
|Nighthawks|            15.25|
+----------+-----------------+



### Step 5. Present general statistics by company

In [25]:
regiments.groupBy('company').avg().show()
regiments.groupBy('company').count().show()
regiments.groupBy('company').sum().show()
regiments.groupBy('company').max().show()
regiments.groupBy('company').min().show()

+-------+-----------------+------------------+
|company|avg(preTestScore)|avg(postTestScore)|
+-------+-----------------+------------------+
|    2nd|             15.5|              67.0|
|    1st|6.666666666666667|57.666666666666664|
+-------+-----------------+------------------+

+-------+-----+
|company|count|
+-------+-----+
|    2nd|    6|
|    1st|    6|
+-------+-----+

+-------+-----------------+------------------+
|company|sum(preTestScore)|sum(postTestScore)|
+-------+-----------------+------------------+
|    2nd|               93|               402|
|    1st|               40|               346|
+-------+-----------------+------------------+

+-------+-----------------+------------------+
|company|max(preTestScore)|max(postTestScore)|
+-------+-----------------+------------------+
|    2nd|               31|                94|
|    1st|               24|                94|
+-------+-----------------+------------------+

+-------+-----------------+------------------+
|compan

### Step 6. What is the mean of each company's preTestScore?

In [27]:
regiments.groupBy('company').avg('preTestScore').show()

+-------+-----------------+
|company|avg(preTestScore)|
+-------+-----------------+
|    2nd|             15.5|
|    1st|6.666666666666667|
+-------+-----------------+



### Step 7. Present the mean preTestScores grouped by regiment and company

In [39]:
means = regiments.groupBy('regiment','company').avg('preTestScore')


+----------+----+----+
|  regiment| 1st| 2nd|
+----------+----+----+
|Nighthawks|14.0|16.5|
|  Dragoons| 3.5|27.5|
|    Scouts| 2.5| 2.5|
+----------+----+----+



### Step 8. Present the mean preTestScores grouped by regiment and company without heirarchical indexing

In [40]:
means.groupBy('regiment').pivot('company').avg().show()

+----------+----+----+
|  regiment| 1st| 2nd|
+----------+----+----+
|Nighthawks|14.0|16.5|
|  Dragoons| 3.5|27.5|
|    Scouts| 2.5| 2.5|
+----------+----+----+



### Step 9. Group the entire dataframe by regiment and company

In [30]:
grouped = regiments.groupBy('regiment','company')

### Step 10. What is the number of observations in each regiment and company

In [31]:
grouped.count().show()

+----------+-------+-----+
|  regiment|company|count|
+----------+-------+-----+
|Nighthawks|    1st|    2|
|  Dragoons|    1st|    2|
|Nighthawks|    2nd|    2|
|  Dragoons|    2nd|    2|
|    Scouts|    2nd|    2|
|    Scouts|    1st|    2|
+----------+-------+-----+



### Step 11. Iterate over a group and print the name and the whole data from the regiment

In [83]:
groups = regiments.groupBy('regiment').count().select('regiment').collect()

for row in groups:
  print(regiments.filter(regiments.regiment == row['regiment']).show())

+----------+-------+--------+------------+-------------+
|  regiment|company|    name|preTestScore|postTestScore|
+----------+-------+--------+------------+-------------+
|Nighthawks|    1st|  Miller|           4|           25|
|Nighthawks|    1st|Jacobson|          24|           94|
|Nighthawks|    2nd|     Ali|          31|           57|
|Nighthawks|    2nd|  Milner|           2|           62|
+----------+-------+--------+------------+-------------+

None
+--------+-------+------+------------+-------------+
|regiment|company|  name|preTestScore|postTestScore|
+--------+-------+------+------------+-------------+
|Dragoons|    1st| Cooze|           3|           70|
|Dragoons|    1st| Jacon|           4|           25|
|Dragoons|    2nd|Ryaner|          24|           94|
|Dragoons|    2nd|  Sone|          31|           57|
+--------+-------+------+------------+-------------+

None
+--------+-------+-----+------------+-------------+
|regiment|company| name|preTestScore|postTestScore|
+---